In [ ]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Mount GDrive to /content/drive</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive!
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import re
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel, BertConfig, TFBertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score

In [ ]:
def tokenize_movies(review, length):
    return tokenizer.encode_plus(
        review,
        add_special_tokens=True,
        max_length=length,
        truncation=True,
        padding='max_length',
        return_attention_mask=True
    )

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/data/nlp-gold/imdb/include_negotions/'

files = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
print(files)

df_imdb = pd.concat((pd.read_csv(f) for f in files), ignore_index=True)

['/content/drive/MyDrive/Colab Notebooks/data/nlp-gold/imdb/include_negotions/imdb_data_0.csv.gz', '/content/drive/MyDrive/Colab Notebooks/data/nlp-gold/imdb/include_negotions/imdb_data_1.csv.gz', '/content/drive/MyDrive/Colab Notebooks/data/nlp-gold/imdb/include_negotions/imdb_data_2.csv.gz', '/content/drive/MyDrive/Colab Notebooks/data/nlp-gold/imdb/include_negotions/imdb_data_3.csv.gz', '/content/drive/MyDrive/Colab Notebooks/data/nlp-gold/imdb/include_negotions/imdb_data_4.csv.gz', '/content/drive/MyDrive/Colab Notebooks/data/nlp-gold/imdb/include_negotions/imdb_data_5.csv.gz', '/content/drive/MyDrive/Colab Notebooks/data/nlp-gold/imdb/include_negotions/imdb_data_6.csv.gz', '/content/drive/MyDrive/Colab Notebooks/data/nlp-gold/imdb/include_negotions/imdb_data_7.csv.gz', '/content/drive/MyDrive/Colab Notebooks/data/nlp-gold/imdb/include_negotions/imdb_data_8.csv.gz', '/content/drive/MyDrive/Colab Notebooks/data/nlp-gold/imdb/include_negotions/imdb_data_9.csv.gz']


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
df_rt = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/nlp-gold/rottentomatoes/include_negotions/rt_data.csv.gz')


<ipython-input-9-194d58e50361>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_rt = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/nlp-gold/rottentomatoes/include_negotions/rt_data.csv.gz')


In [ ]:
df_rt = pd.read_hdf('/content/drive/MyDrive/Colab Notebooks/data/nlp-gold/rottentomatoes/rt_data_tokenized.h5')

In [ ]:
df_rt.dropna(subset=['review_detail'], inplace=True)
df_rt.dropna(subset=['rating'], inplace=True)

In [ ]:
df_imdb.dropna(subset=['review_detail'], inplace=True)
df_imdb.dropna(subset=['rating'], inplace=True)

In [ ]:
df_imdb.head()

In [ ]:
df_rt.head()

,rating,movie,word_count,input_ids,token_type_ids,attention_mask,label_sentiment,label_top_critic
3,7.0,12 Angry Men (Twelve Angry Men),37,"[101, 3251, 9501, 2131, 2369, 7407, 12383, 252...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1,0
6,2.0,The 39 Steps,17,"[101, 4302, 10693, 7324, 27475, 2123, 2102, 22...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0,0
7,7.0,3:10 to Yuma,23,"[101, 11312, 4027, 3475, 2102, 2307, 3185, 220...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1,1
8,8.0,Charly (A Heartbeat Away),4,"[101, 4569, 28022, 28575, 102, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2,0
9,6.0,Abraham Lincoln,30,"[101, 13675, 27479, 8626, 2275, 6155, 13181, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1,1


In [ ]:
print(len(df_rt))

757050


In [ ]:
df_rt['review_detail'] = df_rt['review_detail'].astype("string")

In [ ]:
df_imdb['review_detail'] = df_imdb['review_detail'].astype("string")

In [ ]:
max_length_imdb = df_imdb['review_detail'].apply(lambda x: len(x) if pd.notna(x) else 0).max()

print(max_length_imdb)

In [ ]:
max_length_rt = df_rt['review_detail'].apply(lambda x: len(x) if pd.notna(x) else 0).max()

print(max_length_rt)

252


In [ ]:
df_rt['review_detail'] = df_rt['review_detail'].apply(lambda x: tokenize_movies(x, max_length_rt))

In [ ]:
df_imdb['review_detail'] = df_imdb['review_detail'].apply(lambda x: tokenize_movies(x, max_length_imdb))

In [ ]:
df_rt.head()

,rating,movie,word_count,input_ids,token_type_ids,attention_mask,label_sentiment,label_top_critic
3,7.0,12 Angry Men (Twelve Angry Men),37,"[101, 3251, 9501, 2131, 2369, 7407, 12383, 252...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1,0
6,2.0,The 39 Steps,17,"[101, 4302, 10693, 7324, 27475, 2123, 2102, 22...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0,0
7,7.0,3:10 to Yuma,23,"[101, 11312, 4027, 3475, 2102, 2307, 3185, 220...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1,1
8,8.0,Charly (A Heartbeat Away),4,"[101, 4569, 28022, 28575, 102, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2,0
9,6.0,Abraham Lincoln,30,"[101, 13675, 27479, 8626, 2275, 6155, 13181, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1,1


In [ ]:
df_imdb['input_ids'] = df_imdb['review_detail'].apply(lambda x: x['input_ids'])
df_imdb['token_type_ids'] = df_imdb['review_detail'].apply(lambda x: x['token_type_ids'])
df_imdb['attention_mask'] = df_imdb['review_detail'].apply(lambda x: x['attention_mask'])
df_imdb['label_sentiment'] = df_imdb['sentiment'].apply(lambda x: 0 if x == "NEGATIVE" else 1 if x == "MEDIUM" else 2 if x == "POSITIVE" else -1)
df_imdb = df_imdb.drop('review_detail', axis=1)
df_imdb = df_imdb.drop('sentiment', axis=1)

In [ ]:
print(df_imdb['token_type_ids'].dtype)

object


In [ ]:
df_rt['input_ids'] = df_rt['review_detail'].apply(lambda x: x['input_ids'])
df_rt['token_type_ids'] = df_rt['review_detail'].apply(lambda x: x['token_type_ids'])
df_rt['attention_mask'] = df_rt['review_detail'].apply(lambda x: x['attention_mask'])
df_rt['label_sentiment'] = df_rt['sentiment'].apply(lambda x: 0 if x == "NEGATIVE" else 1 if x == "MEDIUM" else 2 if x == "POSITIVE" else -1)
df_rt['label_top_critic'] = df_rt['top_critic'].apply(lambda x: 0 if x == False else 1 if x == True else -1)
df_rt = df_rt.drop('review_detail', axis=1)
df_rt = df_rt.drop('sentiment', axis=1)
df_rt = df_rt.drop('top_critic', axis=1)

In [ ]:
df_rt.head()

,rating,movie,word_count,input_ids,token_type_ids,attention_mask,label_sentiment,label_top_critic
3,7.0,12 Angry Men (Twelve Angry Men),37,"[101, 3251, 9501, 2131, 2369, 7407, 12383, 252...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1,0
6,2.0,The 39 Steps,17,"[101, 4302, 10693, 7324, 27475, 2123, 2102, 22...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0,0
7,7.0,3:10 to Yuma,23,"[101, 11312, 4027, 3475, 2102, 2307, 3185, 220...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1,1
8,8.0,Charly (A Heartbeat Away),4,"[101, 4569, 28022, 28575, 102, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2,0
9,6.0,Abraham Lincoln,30,"[101, 13675, 27479, 8626, 2275, 6155, 13181, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1,1


In [ ]:
df_imdb.head(1000)

,movie,rating,word_count,input_ids,token_type_ids,attention_mask,label_sentiment
843467,Saints and Soldiers,9.0,43,"[101, 9845, 2408, 9733, 3539, 2678, 15936, 204...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2
3439151,Lost: There's No Place Like Home: Part 2 Seas...,9.0,433,"[101, 2439, 3539, 2442, 18866, 2547, 3342, 203...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2
3326457,The Diving Bell and the Butterfly,9.0,985,"[101, 4326, 2736, 15252, 24659, 2028, 3408, 21...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2
720241,Wizards,9.0,132,"[101, 2048, 2235, 10821, 2186, 1015, 18772, 27...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2
3831072,An American Haunting,8.0,151,"[101, 2137, 20161, 3594, 12472, 5469, 2143, 45...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2
...,...,...,...,...,...,...,...
244587,Kingdom,10.0,214,"[101, 10166, 2265, 4110, 2034, 2792, 2123, 210...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2
249984,The Midnight Gospel: Mouse of Silver Season 1...,10.0,184,"[101, 5263, 3422, 9715, 7861, 25940, 2529, 202...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2
1054102,Onward,9.0,42,"[101, 2081, 5390, 1017, 2335, 2440, 7603, 2802...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2
2442646,Seabiscuit,10.0,79,"[101, 2589, 3652, 8086, 6919, 3276, 2738, 5866...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2


In [ ]:
df_rt.to_hdf('/content/drive/MyDrive/Colab Notebooks/data/nlp-gold/rottentomatoes/rt_data_tokenized.h5', key='df', mode='w')

<ipython-input-28-7822bbc6d7fe>:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['movie', 'input_ids', 'token_type_ids', 'attention_mask'], dtype='object')]

  df_rt.to_hdf('/content/drive/MyDrive/Colab Notebooks/data/nlp-gold/rottentomatoes/rt_data_tokenized.h5', key='df', mode='w')


In [ ]:
df_imdb.to_hdf('/content/drive/MyDrive/Colab Notebooks/data/nlp-gold/imdb/imdb_data_tokenized.h5', key='df', mode='w')

<ipython-input-19-cfe2b084b161>:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['movie', 'input_ids', 'token_type_ids', 'attention_mask'], dtype='object')]

  df_imdb.to_hdf('/content/drive/MyDrive/Colab Notebooks/data/nlp-gold/imdb/imdb_data_tokenized.h5', key='df', mode='w')


# Sampling Data
Choose from multiple Sampling Techniques


In [ ]:
df_imdb_positive = df_imdb[df_imdb['sentiment'] == "POSITIVE"]
df_imdb_negative = df_imdb[df_imdb['sentiment'] == "NEGATIVE"]
df_imdb_medium = df_imdb[df_imdb['sentiment'] == "MEDIUM"]

In [ ]:
print("IMDB POS: "+ str(len(df_imdb_positive)))
print("IMDB NEG: "+ str(len(df_imdb_negative)))
print("IMDB MED: "+ str(len(df_imdb_medium)))

IMDB POS: 2454660
IMDB NEG: 929778
IMDB MED: 1524079


In [ ]:
num_samples_per_class = 20000 // 3

df_imdb_positive = df_imdb_positive.sample(n=num_samples_per_class, random_state=42)
df_imdb_negative = df_imdb_negative.sample(n=num_samples_per_class, random_state=42)
df_imdb_medium = df_imdb_medium.sample(n=num_samples_per_class + 1, random_state=42)


In [ ]:
df_imdb = pd.concat([df_imdb_positive, df_imdb_negative, df_imdb_medium])

In [ ]:
df_imdb.head()

,movie,rating,review_detail,word_count,sentiment
843467,Saints and Soldiers,9.0,stumbled across amazon prime video delighted t...,43,POSITIVE
3439151,Lost: There's No Place Like Home: Part 2 Seas...,9.0,lost prime mustwatch television remember first...,433,POSITIVE
3326457,The Diving Bell and the Butterfly,9.0,strange finished reviewing magnolia one terms ...,985,POSITIVE
720241,Wizards,9.0,two small complaints series 1 abrupt start beg...,132,POSITIVE
3831072,An American Haunting,8.0,american haunting uses conventions horror film...,151,POSITIVE


# Tokenzing IMDB

In [ ]:
df_imdb = pd.read_hdf('/content/drive/MyDrive/Colab Notebooks/data/nlp-gold/imdb/imdb_data_tokenized.h5')

In [ ]:
df_imdb.head()

,movie,rating,word_count,input_ids,token_type_ids,attention_mask,label_sentiment
843467,Saints and Soldiers,9.0,43,"[101, 9845, 2408, 9733, 3539, 2678, 15936, 204...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2
3439151,Lost: There's No Place Like Home: Part 2 Seas...,9.0,433,"[101, 2439, 3539, 2442, 18866, 2547, 3342, 203...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2
3326457,The Diving Bell and the Butterfly,9.0,985,"[101, 4326, 2736, 15252, 24659, 2028, 3408, 21...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2
720241,Wizards,9.0,132,"[101, 2048, 2235, 10821, 2186, 1015, 18772, 27...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2
3831072,An American Haunting,8.0,151,"[101, 2137, 20161, 3594, 12472, 5469, 2143, 45...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2


In [ ]:
print(df_imdb.shape)

(19999, 7)


In [ ]:
train_data_imdb, test_data_imdb = train_test_split(df_imdb, test_size=0.2)

In [ ]:
input_ids_list_train_imdb = train_data_imdb['input_ids'].tolist()
attention_mask_list_train_imdb = train_data_imdb['attention_mask'].tolist()
token_type_ids_list_train_imdb = train_data_imdb['token_type_ids'].tolist()
labels_list_train_imdb = train_data_imdb['label_sentiment'].tolist()

In [ ]:
print(labels_list_train_imdb)

[0, 0, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 0, 1, 2, 1, 0, 2, 0, 0, 1, 2, 1, 0, 1, 1, 0, 2, 1, 2, 0, 1, 0, 2, 2, 1, 2, 1, 0, 2, 2, 0, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 1, 1, 0, 2, 2, 0, 2, 2, 0, 2, 0, 2, 0, 1, 1, 2, 0, 2, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 2, 1, 1, 2, 2, 1, 2, 2, 2, 0, 1, 2, 0, 1, 1, 0, 1, 0, 2, 0, 0, 1, 0, 2, 1, 2, 2, 2, 0, 2, 0, 0, 0, 2, 2, 1, 2, 1, 0, 2, 2, 2, 0, 2, 0, 1, 2, 1, 1, 2, 0, 1, 2, 1, 1, 2, 0, 2, 2, 2, 1, 0, 0, 0, 2, 2, 1, 1, 1, 2, 2, 2, 0, 0, 1, 1, 0, 1, 0, 2, 2, 1, 1, 2, 2, 1, 1, 2, 1, 2, 0, 1, 2, 1, 1, 0, 1, 2, 0, 0, 1, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 1, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 2, 0, 1, 0, 0, 2, 2, 1, 1, 1, 0, 1, 2, 2, 2, 0, 1, 0, 1, 2, 2, 0, 0, 1, 1, 0, 2, 2, 0, 2, 0, 1, 1, 0, 0, 1, 0, 2, 1, 1, 0, 1, 0, 1, 2, 1, 2, 0, 0, 1, 1, 1, 2, 1, 2, 2, 1, 1, 0, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 2, 2, 2, 2, 0, 2, 2, 0, 0, 1, 2, 2, 0, 0, 1, 0, 2, 2, 1, 2, 0, 1, 0, 2, 2, 2, 2, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 

In [ ]:
input_ids_list_test_imdb = test_data_imdb['input_ids'].tolist()
attention_mask_lis_test_imdb = test_data_imdb['attention_mask'].tolist()
token_type_ids_list_test_imdb = test_data_imdb['token_type_ids'].tolist()
labels_list_test_imdb = test_data_imdb['label_sentiment'].tolist()

In [ ]:
input_ids_train_imdb = tf.convert_to_tensor(input_ids_list_test_imdb)
attention_mask_train_imdb = tf.convert_to_tensor(attention_mask_lis_test_imdb)
token_type_ids_train_imdb = tf.convert_to_tensor(token_type_ids_list_test_imdb)
labels_train_imdb = tf.convert_to_tensor(labels_list_test_imdb)

In [ ]:
input_ids_test_imdb = tf.convert_to_tensor(input_ids_list_test_imdb)
attention_mask_test_imdb = tf.convert_to_tensor(attention_mask_lis_test_imdb)
token_type_ids_test_imdb = tf.convert_to_tensor(token_type_ids_list_test_imdb)
labels_test_imdb = tf.convert_to_tensor(labels_list_test_imdb)

In [ ]:
input_ids_test_imdb.shape

TensorShape([4000, 6990])

In [ ]:
dataset_test_imdb = tf.data.Dataset.from_tensor_slices(((input_ids_test_imdb, attention_mask_test_imdb, token_type_ids_test_imdb), labels_test_imdb))
dataset_test_imdb = dataset_test_imdb.batch(16)  # Adjust batch size as needed
dataset_test_imdb = dataset_test_imdb.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
dataset_train_imdb = tf.data.Dataset.from_tensor_slices(((input_ids_test_imdb, attention_mask_test_imdb, token_type_ids_test_imdb), labels_test_imdb))
dataset_train_imdb = dataset_train_imdb.batch(16)  # Adjust batch size as needed
dataset_train_imdb = dataset_train_imdb.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
model.fit(dataset_train_imdb, epochs=1)  # Adjust the number of epochs as needed

ResourceExhaustedError: Graph execution error:

Detected at node tf_bert_for_sequence_classification_1/bert/encoder/layer_._0/attention/self/MatMul defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/io

# Taining the Model

In [ ]:
df_rt = pd.read_hdf('/content/drive/MyDrive/Colab Notebooks/data/nlp-gold/rottentomatoes/rt_data_tokenized.h5')

In [ ]:
train_data, test_data = train_test_split(df_rt, test_size=0.2)

In [ ]:
input_ids_list_train = train_data['input_ids'].tolist()
attention_mask_list_train = train_data['attention_mask'].tolist()
token_type_ids_list_train = train_data['token_type_ids'].tolist()
labels_list_train = train_data['label_top_critic'].tolist()

In [ ]:
input_ids_list_test = test_data['input_ids'].tolist()
attention_mask_lis_test = test_data['attention_mask'].tolist()
token_type_ids_list_test = test_data['token_type_ids'].tolist()
labels_list_test = test_data['label_top_critic'].tolist()

In [ ]:
input_ids_train = tf.convert_to_tensor(input_ids_list_train)
attention_mask_train = tf.convert_to_tensor(attention_mask_list_train)
token_type_ids_train = tf.convert_to_tensor(token_type_ids_list_train)
labels_train = tf.convert_to_tensor(labels_list_train)

In [ ]:
input_ids_test = tf.convert_to_tensor(input_ids_list_test)
attention_mask_test = tf.convert_to_tensor(attention_mask_lis_test)
token_type_ids_test = tf.convert_to_tensor(token_type_ids_list_test)
labels_test = tf.convert_to_tensor(labels_list_test)

In [ ]:
dataset_test = tf.data.Dataset.from_tensor_slices(((input_ids_test, attention_mask_test, token_type_ids_test), labels_test))
dataset_test = dataset_test.batch(32)  # Adjust batch size as needed
dataset_test = dataset_test.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
dataset_train = tf.data.Dataset.from_tensor_slices(((input_ids_train, attention_mask_train, token_type_ids_train), labels_train))
dataset_train = dataset_train.batch(32)  # Adjust batch size as needed
dataset_train = dataset_train.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
model.fit(dataset_train, epochs=1)  # Adjust the number of epochs as needed

18927/18927 [==============================] - 9911s 521ms/step - loss: 0.5477 - accuracy: 0.7647


In [ ]:
loss, accuracy = model.evaluate(dataset_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

   77/18927 [..............................] - ETA: 57:24 - loss: 0.5548 - accuracy: 0.7569

KeyboardInterrupt: 

In [ ]:
model.save_pretrained('/content/drive/MyDrive/Colab Notebooks/ ')

In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/data/nlp_rt', save_format='tf')  # Saves the model in HDF5 format

# Load and Test

In [ ]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss)

In [ ]:
model = TFBertForSequenceClassification.from_pretrained('/content/drive/MyDrive/Colab Notebooks/model_top_critic')

Some layers from the model checkpoint at /content/drive/MyDrive/Colab Notebooks/model_top_critic were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/Colab Notebooks/model_top_critic.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.

In [ ]:
predictions = model.predict(dataset_test)

4732/4732 [==============================] - 865s 182ms/step


In [ ]:
print(predictions)

TFSequenceClassifierOutput(loss=None, logits=array([[ 0.6940142 , -0.48428416],
       [ 0.69401425, -0.48428422],
       [ 0.69401425, -0.48428422],
       ...,
       [ 0.6940143 , -0.48428416],
       [ 0.6940142 , -0.48428416],
       [ 0.6940143 , -0.48428416]], dtype=float32), hidden_states=None, attentions=None)


In [ ]:
logits = np.array(predictions)

In [ ]:
print(logits)

TFSequenceClassifierOutput(loss=None, logits=array([[ 0.6940142 , -0.48428416],
       [ 0.69401425, -0.48428422],
       [ 0.69401425, -0.48428422],
       ...,
       [ 0.6940143 , -0.48428416],
       [ 0.6940142 , -0.48428416],
       [ 0.6940143 , -0.48428416]], dtype=float32), hidden_states=None, attentions=None)


In [ ]:
predicted_classes = np.argmax(logits, axis=1)

AxisError: axis 1 is out of bounds for array of dimension 1

In [ ]:
true_values = []

# Durchlaufen des Datensatzes und Extrahieren der Labels
for _, label in dataset_test:
    true_values.append(label.numpy())

In [ ]:
print(true_values)

[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32), array([0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 0, 0], dtype=int32), array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 1, 0, 0, 1, 1, 0], dtype=int32), array([0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0], dtype=int32), array([1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 0], dtype=int32), array([0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0], dtype=int32), array([0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0], dtype=int32), array([0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1], dtype=int32),

In [ ]:
print(len(predictions))
print(len(true_values))

1
4732


In [ ]:
f1 = f1_score(true_values, predictions, average='weighted')

conf_matrix = confusion_matrix(true_values, predictions)

ValueError: Found input variables with inconsistent numbers of samples: [4732, 1]

In [ ]:
print(f"F1 Score: {f1}")
print("Konfusionsmatrix:")
print(conf_matrix)

In [ ]:
predictions = model.predict(dataset_test_imdb)
probabilities = tf.nn.softmax(predictions.logits, axis=-1)
predicted_labels = np.argmax(probabilities, axis=1)
print(predicted_labels)

ResourceExhaustedError: Graph execution error:

Detected at node tf_bert_for_sequence_classification_1/bert/encoder/layer_._0/attention/self/MatMul defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/io